# Tandem result visualization tools
Last modification: 2023.06.13.

In [2]:
import numpy as np
import change_params
import myplots

mp = myplots.Figpref()
ch = change_params.variate()

prefix = 'Thakur20_various_fractal_profiles/v6_ab2_Dc1_long'

# ----------
save_dir = 'models/'+prefix
plot_dir = 'plots/'+prefix

print('Load saved data: %s/outputs'%(save_dir))
outputs = np.load('%s/outputs.npy'%(save_dir))
print('Load saved data: %s/outputs_depthinfo'%(save_dir))
dep = np.load('%s/outputs_depthinfo.npy'%(save_dir))

# ----------
y,Hs,a,b,a_b,tau0,sigma0,Dc,others = ch.load_parameter(prefix)

Load saved data: models/Thakur20_various_fractal_profiles/v6_ab2_Dc1_long/outputs
Load saved data: models/Thakur20_various_fractal_profiles/v6_ab2_Dc1_long/outputs_depthinfo
Fractal normal stress model ver.6
Fractal a-b model ver.2
Fractal Dc model ver.1
Using file fractal_snpre_06
Using file fractal_ab_02
Using file fractal_Dc_01


# Fault output vs. Time

In [ ]:
plot_in_sec = 0
sliprate_dep,slip_dep,stress_dep,state_var_dep = 7.5,0,0,0
save_on = 0

if abs(sliprate_dep)>0:
    from faultoutputs_vs_time import sliprate_time
    sliprate_time(plot_dir,outputs,dep,sliprate_dep,plot_in_sec,save_on)
    
if abs(slip_dep)>0:
    from faultoutputs_vs_time import slip_time
    slip_time(plot_dir,outputs,dep,slip_dep,plot_in_sec,save_on)
    
if abs(stress_dep)>0:
    from faultoutputs_vs_time import stress_time
    stress_time(plot_dir,outputs,dep,stress_dep,plot_in_sec,save_on)

if abs(state_var_dep)>0:
    from faultoutputs_vs_time import state_time
    state_time(plot_dir,outputs,dep,state_var_dep,plot_in_sec,save_on)

# Check initial inputs

In [ ]:
save_on = 0
stressprof,abprof,dcprof = 1,0,0
if stressprof:
    from stress_profile import plot_stress_vs_depth
    plot_stress_vs_depth(plot_dir,prefix,outputs,dep,save_on)

if abprof:
    from ab_profile import plot_ab_vs_depth
    plot_ab_vs_depth(plot_dir,prefix,save_on)

if dcprof:
    from Dc_profile import plot_Dc_vs_depth
    plot_Dc_vs_depth(plot_dir,prefix,save_on)

## Stress histogram

In [ ]:
save_on = 0
from stress_profile import plot_hist
plot_hist(plot_dir,outputs,dep,save_on)

# Faultoutput as image

In [ ]:
save_on = 0
image_sliprate,image_shearT,image_normalT,image_state_var = 1,0,0,0
vmin,vmax = None,None
if vmin is None and image_sliprate:
    vmin = 1e-13
plot_in_timestep = 1
plot_in_sec = 0
from faultoutputs_image import fout_image
fout_image(image_sliprate,image_shearT,image_normalT,image_state_var,outputs,dep,plot_in_timestep,plot_dir,prefix,vmin,vmax,plot_in_sec,save_on)


# Cumslip vs. Depth

In [ ]:
from cumslip_compute import *
from cumslip_plot import *

rths = 10
save_on = 0
spin_up = 0
ab_inout,stress_inout,dc_inout,depth_dist = 1,0,1,0

if prefix == 'BP1':
    Vlb = 1e-2
    dt_creep = 5*ch.yr2sec
    dt_coseismic = 1
    dt_interm = 1*ch.wk2sec
    cuttime = 1200*ch.yr2sec
else:
    Vths = 1e-2
    Vlb = 0
    dt_creep = 2*ch.yr2sec
    dt_coseismic = 0.5
    dt_interm = 0
    cuttime = 0
    mingap = 60

# ----------
cumslip_outputs = compute_cumslip(outputs,dep,cuttime,Vlb,Vths,dt_creep,dt_coseismic,dt_interm,mingap)

# --- Plot the result
if spin_up > 0:
    spup_cumslip_outputs = compute_spinup(outputs,dep,cuttime,cumslip_outputs,spin_up)
    spup_where(plot_dir,prefix,cumslip_outputs,spup_cumslip_outputs,Vths,dt_coseismic,rths,save_on)
else:
    spup_cumslip_outputs = None

In [ ]:
if sum([ab_inout,stress_inout,dc_inout,depth_dist]) == 0:
    only_cumslip(save_dir,prefix,cumslip_outputs,Vths,dt_coseismic,rths,spup_cumslip_outputs)
elif sum([ab_inout,stress_inout,dc_inout,depth_dist]) == 1:
    two_set(save_dir,prefix,outputs,dep,cumslip_outputs,Vths,dt_coseismic,depth_dist,ab_inout,stress_inout,dc_inout,rths,spup_cumslip_outputs)
elif sum([ab_inout,stress_inout,dc_inout,depth_dist]) == 2:
    three_set(save_dir,prefix,outputs,dep,cumslip_outputs,Vths,dt_coseismic,depth_dist,ab_inout,stress_inout,dc_inout,rths,spup_cumslip_outputs)

In [ ]:
plot_event_analyze(plot_dir,prefix,cumslip_outputs,rths,save_on)

# Source-time function
After Barbot (2019)

In [ ]:
from scipy import integrate

time = np.array([outputs[i][:,0] for i in np.argsort(abs(dep))])
sr = abs(np.array([outputs[i][:,4] for i in np.argsort(abs(dep))]))
z = np.sort(abs(dep))

mu = 10e9

Fdot = mu * integrate.simpson(sr,z)

# Recurrence interval

In [ ]:
from cumslip_compute import *
from cumslip_plot import *

rths = 10
save_on = 0
spin_up = 1
ab_inout,stress_inout,dc_inout,depth_dist = 1,0,1,0

if prefix == 'BP1':
    Vlb = 1e-2
    dt_creep = 5*ch.yr2sec
    dt_coseismic = 1
    dt_interm = 1*ch.wk2sec
    cuttime = 1200*ch.yr2sec
else:
    Vths = 1e-2
    Vlb = 0
    dt_creep = 2*ch.yr2sec
    dt_coseismic = 0.5
    dt_interm = 0
    cuttime = 0
    mingap = 60

# ----------
from cumslip_compute import *
from cumslip_plot import *

cumslip_outputs = compute_cumslip(outputs,dep,cuttime,Vlb,Vths,dt_creep,dt_coseismic,dt_interm,mingap,print_on=False)

system_wide = analyze_events(cumslip_outputs,rths)[2]
sys_ev_t = cumslip_outputs[0][0][system_wide]
sys_ev_t = sys_ev_t[cumslip_outputs[1][0][system_wide]>spin_up]

nmodes=2
print('Recurrence interval')
for m in range(nmodes):
    print('Mode %d: %2.3f yrs'%(m+1,np.mean(np.diff(sys_ev_t[m::nmodes]))/ch.yr2sec))


# Phase diagram

In [ ]:
save_on = 0
Vths = 1e-2

idx = np.argsort(abs(dep))
sorted_outputs = np.array(np.copy(outputs[idx]))

sliprate = sorted_outputs[:,:,4].transpose()
shear_s = abs(sorted_outputs[:,:,3]).transpose()
normal_s = sorted_outputs[:,:,5].transpose()

dx = np.diff(np.sort(abs(dep)))
delx = np.tile(dx,shear_s.shape[0]).reshape(shear_s.shape[0],len(dx))
shear_to_normal = shear_s/normal_s
int_shear_to_normal = np.sum(0.5*delx*(shear_to_normal[:,0:-1]+shear_to_normal[:,1:]),axis=1)
int_sliprate = np.sum(0.5*delx*(sliprate[:,0:-1]+sliprate[:,1:]),axis=1)

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(8,6))
# stp = int(1e4)
stp = int(1.5e4)
c =0
# startp = 0
startp = int(3e4)
# if startp == 0:
#     plt.title('From %d, step = %1.1e'%(startp,stp),fontsize=20,fontweight = 'bold')
# else:
#     plt.title('From %1.1e, step = %1.1e'%(startp,stp),fontsize=20,fontweight = 'bold')
# while startp <= len(int_sliprate):
#     if startp + stp <= len(int_sliprate):
#         plt.plot(int_sliprate[startp:startp+stp],int_shear_to_normal[startp:startp+stp],lw=3,label='%d'%(c))
#     else:
#         plt.plot(int_sliprate[startp:],int_shear_to_normal[startp:],lw=3,label='%d'%(c))
#     startp += stp
#     c += 1

i1 = 10
i2 = 40
plt.plot(int_sliprate[startp:],int_shear_to_normal[startp:],lw=3,color='0.62',label='All')
evidx = np.zeros(len(int_sliprate[startp:]))
evidx[np.where(np.sum(sliprate[startp:,:]>Vths,axis=1)>0)[0]] = np.ones(len(np.where(np.sum(sliprate[startp:,:]>Vths,axis=1)>0)[0]))
evidx = np.array(evidx,dtype=bool)
plt.plot(int_sliprate[startp:][evidx],int_shear_to_normal[startp:][evidx],lw=3,color=mp.mynavy,label='Partial rupture')
plt.plot(int_sliprate[startp:][sliprate[startp:,i1]>Vths],int_shear_to_normal[startp:][sliprate[startp:,i1]>Vths],lw=3,color=mp.myburgundy,label='Whole system rupture')

plt.xlabel('Integrated slip-rate [m/s km]',fontsize=17)
plt.ylabel('Integrated shear stress/normal stress [km]',fontsize=17)
plt.xscale('log')
plt.legend(fontsize=13,loc='lower left')
xl = plt.gca().get_xlim()
yl = plt.gca().get_ylim()

plt.tight_layout()
if save_on:
    plt.savefig('plots/%s/phase_diagram.png'%(prefix),dpi=300)
plt.show()



In [ ]:
import matplotlib.animation as animation 

save_on = 1
Vths = 1e-2

idx = np.argsort(abs(dep))
sorted_outputs = np.array(np.copy(outputs[idx]))

sliprate = sorted_outputs[:,:,4].transpose()
shear_s = abs(sorted_outputs[:,:,3]).transpose()
normal_s = sorted_outputs[:,:,5].transpose()

dx = np.diff(np.sort(abs(dep)))
delx = np.tile(dx,shear_s.shape[0]).reshape(shear_s.shape[0],len(dx))
shear_to_normal = shear_s/normal_s
int_shear_to_normal = np.sum(0.5*delx*(shear_to_normal[:,0:-1]+shear_to_normal[:,1:]),axis=1)
int_sliprate = np.sum(0.5*delx*(sliprate[:,0:-1]+sliprate[:,1:]),axis=1)

plt.rcParams['font.size'] = '15'
fig = plt.figure(figsize=(8,6))

xdata = []   
ydata = []

startp = int(3e4)
def animate(i,int_sliprate,int_shear_to_normal,startp):
    xdata.append(int_sliprate[startp+100*i])
    ydata.append(int_shear_to_normal[startp+i])
    plt.gca().clear()
    plt.plot(xdata,ydata,color='k',lw=3)
    plt.xlabel('Integrated slip-rate [m/s km]',fontsize=17)
    plt.ylabel('Integrated shear stress/normal stress [km]',fontsize=17)
    plt.xlim(2.798309627942856e-09, 10.194717872257138)
    plt.ylim(13.05,13.6)
    plt.xscale('log')
   
anim = animation.FuncAnimation(fig,animate,frames=500,fargs=(int_sliprate,int_shear_to_normal,startp),interval=5,repeat=False) 
if save_on:
    anim.save('plots/%s/phase_diagram.mp4'%(prefix))

In [ ]:
i = 10
print(int_sliprate[startp+i])

## Inidividual

In [ ]:
save_on = 1
target_depth = 12 # in km
indx = np.argmin(abs(abs(dep) - abs(target_depth)))
print('Depth = %1.1f [km]'%abs(dep[indx]))

# mode = 0
# if mode == 1:
#     print('Mode 1: Odd events only')
# elif mode == 2:
#     print('Mode 2: Even events only')
# else:
#     print('Mode 0: All events')

sliprate = outputs[indx,:,4].transpose()
shear_s = abs(outputs[indx,:,3]).transpose()
normal_s = outputs[indx,:,5].transpose()

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(8,7))
# stp = int(1e4)
stp = int(1.5e4)
c =0
startp = 0
# startp = int(3e4)
if startp == 0:
    plt.title('From %d, step = %1.1e'%(startp,stp),fontsize=20,fontweight = 'bold')
else:
    plt.title('From %1.1e, step = %1.1e'%(startp,stp),fontsize=20,fontweight = 'bold')
while startp <= len(sliprate):
    if startp + stp <= len(sliprate):
        plt.plot(abs(sliprate[startp:startp+stp]),shear_s[startp:startp+stp]/normal_s[startp:startp+stp],lw=3,label='%d'%(c))
    else:
        plt.plot(abs(sliprate[startp:]),shear_s[startp:]/normal_s[startp:],lw=3,label='%d'%(c))
    startp += stp
    c += 1

# plt.plot(abs(sliprate),shear_s/normal_s,lw=3)
plt.xlabel('Slip-rate [m/s]',fontsize=17)
plt.ylabel('Shear stress/normal stress',fontsize=17)
# plt.title('Depth = %1.1f [km]'%abs(dep[indx]),fontsize=20,fontweight = 'bold')
plt.xscale('log')
xl = plt.gca().get_xlim()
yl = plt.gca().get_ylim()
plt.text(xl[0]*2,yl[1]-0.025*(yl[1]-yl[0]),'z = %1.1f [km]'%abs(dep[indx]),fontsize=17,fontweight = 'bold',ha='left',va='top',bbox=mp.tboxprop)
plt.legend(fontsize=15,loc='lower left')

plt.tight_layout()
if save_on:
    plt.savefig('plots/%s/indiviaul_phase_diagram_%d.png'%(prefix,abs(dep[indx])*10),dpi=300)
plt.show()



## Several depths together

In [ ]:
def plot_individual(outputs,dep,target_depth,startp,col):
    indx = np.argmin(abs(abs(dep) - abs(target_depth)))
    print('Depth = %1.1f [km]'%abs(dep[indx]))

    sliprate = outputs[indx,:,4].transpose()
    shear_s = abs(outputs[indx,:,3]).transpose()
    normal_s = outputs[indx,:,5].transpose()
    plt.plot(abs(sliprate[startp:]),shear_s[startp:]/normal_s[startp:],lw=2.5,color=col,label='z = %1.1f km'%(abs(dep[indx])))

save_on = 1
startp = int(3e4)

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(8,7))
plot_individual(outputs,dep,2.5,startp,mp.myburgundy)
plot_individual(outputs,dep,8,startp,mp.mygreen)
plot_individual(outputs,dep,12,startp,mp.mynavy)
plt.title('Spun up (From %1.0e)'%(startp),fontsize=20,fontweight = 'bold')
plt.xlabel('Slip-rate [m/s]',fontsize=17)
plt.ylabel('Shear stress/normal stress',fontsize=17)
plt.xscale('log')
xl = plt.gca().get_xlim()
yl = plt.gca().get_ylim()
plt.legend(fontsize=14,loc='upper left')

plt.tight_layout()
if save_on:
    plt.savefig('plots/%s/together_indiviaul_phase_diagram.png'%(prefix),dpi=300)
plt.show()



# Appendix 1. Range of events along the whole depth

In [ ]:
save_on = 1
plot_in_sec = 0

evtimes = []
deps = []

plt.rcParams['font.size'] = '15'
plt.figure(figsize=(12,9))

c = 0
for i in np.argsort(abs(dep)):
    z = abs(dep[i])
    time = np.array(outputs[i])[:,0]
    sliprate = np.array(outputs[i])[:,4]
    cumslip = np.array(outputs[i])[:,2]

    if abs(cuttime) >= 1e-3:
        if cuttime > np.max(time):
            raise ValueError('Cuttime larger than total simulation time - check again')
        else:
            sliprate = sliprate[time <= cuttime]
            cumslip = cumslip[time <= cuttime]
            time = time[time <= cuttime]

    # Define events by sliprate rager than certain threshold
    if Vlb > 0:
        events = np.where(np.logical_and(sliprate < Vths,sliprate > Vlb))[0]
    else:
        events = np.where(sliprate > Vths)[0]

    if len(events) > 0:
        plt.scatter(time[events],z*np.ones(events.shape),s=5,c='k',zorder=3)
        plt.xlabel('Time [s]',fontsize=17)
    elif len(events) == 0:
        print('Depth',z,' - no events')

    c += 1

plt.ylabel('Depth [km]',fontsize=17)
evindx = 25
inc = 0.25
plt.vlines(x=tstart_coseis,ymin=0,ymax=15,colors=mp.myburgundy,lw=3)
plt.vlines(x=tend_coseis,ymin=0,ymax=15,colors=mp.mygreen,lw=3)
# plt.xlim(tstart[evindx]-5,tend[evindx]+5)
# plt.text(tstart[evindx]-4,-0.25,'tstart = %2.1f km'%evdep[evindx],fontsize=15,fontweight='bold',ha='left',va='top',bbox=mp.tboxprop)
plt.xlim(tstart_coseis[evindx]-inc,tend_coseis[evindx]+inc)
plt.text(tstart_coseis[evindx]-(inc-1),-0.25,'tstart = %2.1f km'%evdep[evindx],fontsize=15,fontweight='bold',ha='left',va='top',bbox=mp.tboxprop)
plt.gca().invert_yaxis()
plt.tight_layout()
if save_on:
    plt.savefig('plots/%s/event_range.png'%(prefix),dpi=300)    
plt.show()